可视化

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np
writer = SummaryWriter()
for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
img = np.zeros((3, 100, 100))
img[0] = np.arange(0, 10000).reshape(100, 100) / 10000
img[1] = 1 - np.arange(0, 10000).reshape(100, 100) / 10000

writer.add_image('my_image', img, 0)
writer.close()

In [ ]:
from visdom import Visdom
import numpy as np
import time
from torch import nn
import torch
import random

# python3 -m visdom.server
class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(1))
        self.bias = nn.Parameter(torch.randn(1))
    def forward(self, input):
        return (input * self.weight) + self.bias

w = 2
b = 3
model = LinearModel()
x_train = np.random.randint(low=-10, high=10, size=30)
y_train = [w*x+b+random.randint(0,2) for x in x_train]
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-2, momentum=0.9)
y_train = torch.tensor(y_train, dtype=torch.float32)

viz = Visdom()
viz.line([0.],[0], win='train_loss', opts=dict(title='train_loss'))

for n_iter in range(1000):
    input = torch.from_numpy(x_train)
    output = model(input)
    loss = nn.MSELoss()(output, y_train)
    model.zero_grad()
    loss.backward()
    optimizer.step()
    viz.line([loss.item()], [n_iter], win='train_loss', update='append')

img = np.zeros((3,100,100))
img[0] = np.arange(0, 10000).reshape(100, 100) / 10000
viz.image(img)